In [ ]:
import os

from ase.io import write as ase_write
from ase.optimize import BFGS
from ase import Atoms

from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor

In [ ]:
import os
env_var = "MP_API_KEY"
mp_api_key = os.environ.get(env_var, None)

In [ ]:
mp_id="mp-149"
with MPRester(mp_api_key) as m:
    struct = m.get_structure_by_material_id(mp_id)

In [ ]:
sga = SpacegroupAnalyzer(struct, symprec=1e-4)
conv = sga.get_conventional_standard_structure()

conv_atoms = AseAtomsAdaptor.get_atoms(conv)

In [ ]:
import copy
si_333 = conv_atoms.repeat((3,3,3))
orig_si_333 = copy.deepcopy(si_333)

In [ ]:
ase_write("si333_DATA", si_333, format="lammps-data", masses=True)

In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths="../../models/mace-mpa-0-medium.model", device="cpu")

In [ ]:
si_333.set_calculator(mace_calc)

In [ ]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
temp = 500
MaxwellBoltzmannDistribution(si_333,temperature_K=temp)

In [ ]:
from ase import units
from ase.md.nose_hoover_chain import NoseHooverChainNVT
dyn = NoseHooverChainNVT(si_333, 1.0 * units.fs, temperature_K=temp,
                         tdamp = 100.0)

In [ ]:
from ase.io.trajectory import Trajectory

traj_file = "silicon_mace_medium-mpa.traj"
traj = Trajectory(traj_file, 'w', si_333)
dyn.attach(traj.write, interval=50)

import csv
log_file = "silicon_mace_medium-mpa_500K_md.csv"
# Prepare CSV header
with open(log_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Step", "E_pot (eV)", "E_kin (eV)", "T (K)", "E_tot (eV)"])

# Logging function
def log_status():
    epot = si_333.get_potential_energy()
    ekin = si_333.get_kinetic_energy()
    temp = ekin / (1.5 * units.kB * len(si_333))
    etot = epot + ekin
    step = dyn.nsteps
    print(f"Step {step:6d} | T = {temp:8.1f} K  | "
          f"E_pot = {epot:9.3f} eV  |  E_tot = {etot:9.3f} eV")
    with open(log_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([step, epot, ekin, temp, etot])

dyn.attach(log_status, interval=10)

In [ ]:
dyn.run(5000)